In [1]:
import os
import hddm
import pickle
from kabuki.analyze import gelman_rubin
import kabuki
from IPython.parallel import Client

/Users/minheeyoo/opt/anaconda3/envs/hddmenv/lib/python3.6/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


# Description of model and data 
* z = z0 + z1 * tVD 
* v = v0 + v1 * tVD + v2 * cVD
    * tVD : target value difference (higher value target - lower value target)

In [2]:
data = hddm.load_csv('Model2_data.csv')
data

subj_idx  response       rt trialType       tVD       cVD    tVDcVD
0            1         1  0.58297        hh  0.096667  0.000000  0.000000
1            1         1  0.33312        ll  0.081000 -0.042222 -0.003420
2            1         0  0.48313        Hh  0.086667  1.833333  0.158889
3            1         1  0.86631        Hl  0.234333 -1.888889 -0.442630
4            1         1  0.54919        Hh  0.096667  1.832222  0.177115
...        ...       ...      ...       ...       ...       ...       ...
4085        24         0  0.57208        Hl  0.067333 -2.000000 -0.134667
4086        24         1  0.48706        ll  0.121000  0.000000  0.000000
4087        24         1  0.58487        ll  0.193333  0.000000  0.000000
4088        24         1  0.39946        hh  0.045667  0.073333  0.003349
4089        24         0  0.89197        Hl  0.359000 -1.992222 -0.715208

[4090 rows x 7 columns]

In [3]:
def run_model(i):
    import os
    import hddm
    import pickle
    pathAnalysis = 'project'
    os.chdir(pathAnalysis)
    data = hddm.load_csv(pathAnalysis + '/' + 'Model2_data.csv')
    # Fit model
    v_reg = {'model': 'v ~ 1 + tVD + cVD', 
             'link_func': lambda x: x}
    z_reg = {'model': 'z ~ 1 + tVD', 'link_func': lambda x: x}
    reg_descr = [z_reg, v_reg]
    m = hddm.HDDMRegressor(data, reg_descr,
                           include=('z'), group_only_regressors=False)
    m.sample(10000, burn=4000, dbname='Model2b_chain%i.db'%i, db='pickle')
    return m

In [4]:
v = Client()[:]
jobs = v.map(run_model, range(3)) # 4 is the number of CPUs
model_set = jobs.get()

In [5]:
# save model set
with open('Model2b_hddm_model_set', 'wb') as f:
    pickle.dump(model_set, f)

In [6]:
models = kabuki.utils.concat_models(model_set)

In [7]:
gelman_rubin(model_set)

{'a': 1.0001930746679082,
 'a_std': 1.0000153070290914,
 'a_subj.1': 0.999927503578906,
 'a_subj.2': 1.0000857578999442,
 'a_subj.3': 0.9999631895508426,
 'a_subj.4': 0.9999951657619403,
 'a_subj.5': 1.0005566399697623,
 'a_subj.6': 0.9999589715155826,
 'a_subj.7': 1.0001070156729743,
 'a_subj.8': 1.0005701540513077,
 'a_subj.9': 1.0001157044683853,
 'a_subj.10': 1.0002255989653654,
 'a_subj.11': 1.000805636004395,
 'a_subj.12': 1.0001544984212467,
 'a_subj.13': 1.0000617097247795,
 'a_subj.14': 0.9999348297791638,
 'a_subj.15': 0.99995509221888,
 'a_subj.16': 0.999948626307959,
 'a_subj.17': 0.9999279127880426,
 'a_subj.18': 0.9999957283147469,
 'a_subj.19': 1.000084294511052,
 'a_subj.20': 1.0007508154538636,
 'a_subj.21': 1.0005568573788328,
 'a_subj.22': 0.9999729559129115,
 'a_subj.23': 1.0004686613976925,
 'a_subj.24': 1.0006511249230916,
 't': 1.0001295903565568,
 't_std': 1.000042683915024,
 't_subj.1': 1.0000396136645588,
 't_subj.2': 1.0002526363309647,
 't_subj.3': 1.0000021

In [10]:
import csv
tmp = models.dic_info
with open('Model2b_dic_info.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, tmp.keys())
    w.writeheader()
    w.writerow(tmp)

In [3]:
ppc_data = hddm.utils.post_pred_gen(models, samples = 300)

 [-------------------112%-------------------] 27 of 24 complete in 2755.8 sec

In [4]:
ppc_data.to_csv('Model2b_PPC.csv')

In [ ]:
# save samples
samples = models.get_traces()
samples.to_csv('Model2b_samples.csv')